바이낸스 1분봉 가져오기  
참고자료 : https://wikidocs.net/31065  

환율 바꾸는 코드 자료 : https://appia.tistory.com/434

깃허브 참고 자료 : https://github.com/codeninja/CCXT-Historical-Data

In [1]:
import ccxt

In [7]:
binance = ccxt.binance()
markets = binance.fetch_tickers()

In [11]:
list(markets.keys())[:5]

['ETH/BTC', 'LTC/BTC', 'BNB/BTC', 'NEO/BTC', 'QTUM/ETH']

In [14]:
ticker = binance.fetch_ticker('XRP/USDT')
ticker

{'symbol': 'XRP/USDT',
 'timestamp': 1637562547700,
 'datetime': '2021-11-22T06:29:07.700Z',
 'high': 1.0886,
 'low': 1.0251,
 'bid': 1.0377,
 'bidVolume': 7356.0,
 'ask': 1.0378,
 'askVolume': 60.0,
 'vwap': 1.0624236,
 'open': 1.0794,
 'close': 1.0377,
 'last': 1.0377,
 'previousClose': 1.0794,
 'change': -0.0417,
 'percentage': -3.863,
 'average': 1.0585499999999999,
 'baseVolume': 274989399.0,
 'quoteVolume': 292155226.0216,
 'info': {'symbol': 'XRPUSDT',
  'priceChange': '-0.04170000',
  'priceChangePercent': '-3.863',
  'weightedAvgPrice': '1.06242360',
  'prevClosePrice': '1.07940000',
  'lastPrice': '1.03770000',
  'lastQty': '18.00000000',
  'bidPrice': '1.03770000',
  'bidQty': '7356.00000000',
  'askPrice': '1.03780000',
  'askQty': '60.00000000',
  'openPrice': '1.07940000',
  'highPrice': '1.08860000',
  'lowPrice': '1.02510000',
  'volume': '274989399.00000000',
  'quoteVolume': '292155226.02160000',
  'openTime': '1637476147700',
  'closeTime': '1637562547700',
  'firstI

In [24]:
ohlcvs = binance.fetch_ohlcv('XRP/USDT')
print(ohlcvs[:5])
print('불러온 데이터 개수 :',len(ohlcvs))

[[1637533080000, 1.074, 1.0747, 1.074, 1.0745, 87615.0], [1637533140000, 1.0745, 1.0747, 1.0743, 1.0745, 41888.0], [1637533200000, 1.0745, 1.0748, 1.0743, 1.0744, 33353.0], [1637533260000, 1.0746, 1.0746, 1.0732, 1.0734, 194626.0], [1637533320000, 1.0734, 1.0736, 1.0733, 1.0734, 39745.0]]
불러온 데이터 개수 : 500


In [44]:
from datetime import datetime
for ohlc in ohlcvs[-5:]:
    print(datetime.fromtimestamp(ohlc[0]/1000).strftime('%Y-%m-%d %H:%M:%S'))

2021-11-22 15:33:00
2021-11-22 15:34:00
2021-11-22 15:35:00
2021-11-22 15:36:00
2021-11-22 15:37:00


In [96]:
def scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    earliest_timestamp = exchange.milliseconds() # 현재시간?
    timeframe_duration_in_seconds = exchange.parse_timeframe(timeframe) # 초단위로 바꿈
    timeframe_duration_in_ms = timeframe_duration_in_seconds * 1000
    timedelta = limit * timeframe_duration_in_ms
    if isinstance(since, str):
        since = exchange.parse8601(since)

    all_ohlcv = []
    while True:
        fetch_since = earliest_timestamp - timedelta
        ohlcv = retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, fetch_since, limit)
        # if we have reached the beginning of history
        if ohlcv[0][0] >= earliest_timestamp:
            break
        earliest_timestamp = ohlcv[0][0]
        all_ohlcv = ohlcv + all_ohlcv
        if len(all_ohlcv) % 10000 == 0:
            print(len(all_ohlcv), symbol, 'candles in total from', exchange.iso8601(all_ohlcv[0][0]), 'to', exchange.iso8601(all_ohlcv[-1][0]))
        # if we have reached the checkpoint
        if fetch_since < since:
            break
    return all_ohlcv

def retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    num_retries = 0
    try:
        num_retries += 1
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
        # print('Fetched', len(ohlcv), symbol, 'candles from', exchange.iso8601 (ohlcv[0][0]), 'to', exchange.iso8601 (ohlcv[-1][0]))
        return ohlcv
    except Exception:
        if num_retries > max_retries:
            raise  # Exception('Failed to fetch', timeframe, symbol, 'OHLCV in', max_retries, 'attempts')

In [108]:
binance = ccxt.binance()
all_ohlcv = scrape_ohlcv(binance, 3, 'XRP/USDT', '1m', '2017-01-01 00:00:00Z', 1000)

10000 XRP/USDT candles in total from 2021-11-15T09:24:00.000Z to 2021-11-22T08:03:00.000Z
20000 XRP/USDT candles in total from 2021-11-08T10:44:00.000Z to 2021-11-22T08:03:00.000Z
30000 XRP/USDT candles in total from 2021-11-01T12:04:00.000Z to 2021-11-22T08:03:00.000Z
40000 XRP/USDT candles in total from 2021-10-25T13:24:00.000Z to 2021-11-22T08:03:00.000Z
50000 XRP/USDT candles in total from 2021-10-18T14:44:00.000Z to 2021-11-22T08:03:00.000Z
60000 XRP/USDT candles in total from 2021-10-11T16:04:00.000Z to 2021-11-22T08:03:00.000Z
70000 XRP/USDT candles in total from 2021-10-04T17:24:00.000Z to 2021-11-22T08:03:00.000Z
80000 XRP/USDT candles in total from 2021-09-27T18:44:00.000Z to 2021-11-22T08:03:00.000Z
90000 XRP/USDT candles in total from 2021-09-20T20:04:00.000Z to 2021-11-22T08:03:00.000Z
100000 XRP/USDT candles in total from 2021-09-13T21:24:00.000Z to 2021-11-22T08:03:00.000Z
110000 XRP/USDT candles in total from 2021-09-06T22:44:00.000Z to 2021-11-22T08:03:00.000Z
120000 X

In [4]:
import pandas as pd

dat = pd.DataFrame(all_ohlcv, columns=['time','open','high','low','close','volume'])
dat['time'] = [datetime.fromtimestamp(t/1000).strftime('%Y-%m-%d %H:%M:%S') for t in dat['time']]
dat.to_csv('XRP-USDT.csv', index=False)

In [107]:
dat

,time,open,high,low,close,volume
0,2018-05-04 17:11:00,0.50000,0.50000,0.50000,0.50000,109.51
1,2018-05-04 17:12:00,0.90000,0.92999,0.85000,0.90000,167543.47
2,2018-05-04 17:13:00,0.90000,0.93000,0.89001,0.91000,179750.59
3,2018-05-04 17:14:00,0.91000,1.50000,0.90009,0.92999,107222.48
4,2018-05-04 17:15:00,0.92999,0.95001,0.91020,0.91020,171304.56
...,...,...,...,...,...,...
1870995,2021-11-22 16:24:00,1.04580,1.04620,1.04470,1.04520,291352.00
1870996,2021-11-22 16:25:00,1.04520,1.04730,1.04500,1.04710,334338.00
1870997,2021-11-22 16:26:00,1.04710,1.04820,1.04650,1.04750,255795.00
1870998,2021-11-22 16:27:00,1.04740,1.04860,1.04700,1.04810,178432.00


In [42]:
# pip install finance-datareader 패키지 이름 주의

import FinanceDataReader as fdr
ex_rate = fdr.DataReader('USD/KRW','2018')
ex_rate = ex_rate.reset_index()[['Date','Close']]
ex_rate['Date'] = ex_rate['Date'].astype('str')
ex_rate.head()

,Date,Close
0,2018-01-01,1066.60
1,2018-01-02,1063.21
2,2018-01-03,1064.80
3,2018-01-04,1062.28
4,2018-01-05,1062.09


In [40]:
dat = pd.read_csv('XRP-USDT.csv')
dat['time'] = pd.to_datetime(dat['time'])
dat['ymd'] = dat['time'].dt.date.astype('str')

In [72]:
total_dat = pd.DataFrame()
for i, date in enumerate(pd.unique(dat['ymd'])):
    USD = ex_rate.loc[ex_rate['Date']==pd.unique(dat['ymd'])[0]]['Close']
    temp_dat = dat.loc[dat['ymd']==pd.unique(dat['ymd'])[0]]
    temp_dat.loc[:,['open','high','low','close']] *= float(USD)
    total_dat.append(temp_dat)
    print(date)

C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:1717: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)


2018-05-04
2018-05-05
2018-05-06
2018-05-07
2018-05-08
2018-05-09
2018-05-10
2018-05-11
2018-05-12
2018-05-13
2018-05-14
2018-05-15
2018-05-16
2018-05-17
2018-05-18
2018-05-19
2018-05-20
2018-05-21
2018-05-22
2018-05-23
2018-05-24
2018-05-25
2018-05-26
2018-05-27
2018-05-28
2018-05-29
2018-05-30
2018-05-31
2018-06-01
2018-06-02
2018-06-03
2018-06-04
2018-06-05
2018-06-06
2018-06-07
2018-06-08
2018-06-09
2018-06-10
2018-06-11
2018-06-12
2018-06-13
2018-06-14
2018-06-15
2018-06-16
2018-06-17
2018-06-18
2018-06-19
2018-06-20
2018-06-21
2018-06-22
2018-06-23
2018-06-24
2018-06-25
2018-06-26
2018-06-27
2018-06-28
2018-06-29
2018-06-30
2018-07-01
2018-07-02
2018-07-03
2018-07-04
2018-07-05
2018-07-06
2018-07-07
2018-07-08
2018-07-09
2018-07-10
2018-07-11
2018-07-12
2018-07-13
2018-07-14
2018-07-15
2018-07-16
2018-07-17
2018-07-18
2018-07-19
2018-07-20
2018-07-21
2018-07-22
2018-07-23
2018-07-24
2018-07-25
2018-07-26
2018-07-27
2018-07-28
2018-07-29
2018-07-30
2018-07-31
2018-08-01
2018-08-02

KeyboardInterrupt: 

In [64]:
USD = ex_rate.loc[ex_rate['Date']==pd.unique(dat['ymd'])[0]]['Close']
temp_dat = dat.loc[dat['ymd']==pd.unique(dat['ymd'])[0]]

In [70]:
temp_dat.loc[:,['open','high','low','close']] *= float(USD)

,open,high,low,close
0,6.196070e+08,6.196070e+08,6.196070e+08,6.655260e+11
1,1.115293e+09,1.152457e+09,1.053332e+09,1.197947e+12
2,1.115293e+09,1.152469e+09,1.102913e+09,1.211257e+12
3,1.127685e+09,1.858821e+09,1.115404e+09,1.237865e+12
4,1.152457e+09,1.177266e+09,1.127933e+09,1.211524e+12
...,...,...,...,...
404,1.118663e+09,1.118663e+09,1.116557e+09,1.201554e+12
405,1.118651e+09,1.118725e+09,1.116594e+09,1.199384e+12
406,1.116643e+09,1.118725e+09,1.116631e+09,1.201621e+12
407,1.116804e+09,1.118713e+09,1.116656e+09,1.199424e+12
